In [20]:
import pandas as pd
import os



# Define file path (Professional relative path, works on any computer)


In [21]:
file_path = '../data/raw/online_retail_II.xlsx'



# Load the dataset (Sheet 1 - 2009-2010 data)


In [22]:

print("Loading data...")
df = pd.read_excel(file_path, sheet_name='Year 2009-2010')



Loading data...


# Basic Sanity Check

In [23]:

print("Data loaded successfully!")
print(f"Total Rows: {df.shape[0]}")
print(f"Total Columns: {df.shape[1]}")



Data loaded successfully!
Total Rows: 525461
Total Columns: 8


# Preview data


In [24]:
display(df.head())

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


# 1. Cleaning


In [25]:
df_clean = df.dropna(subset=['Customer ID']).copy()

# Remove cancelled orders (Negative Quantity) & Free items (0 Price)


In [26]:
df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['Price'] > 0)]

# Ensure types


In [27]:
df_clean['Customer ID'] = df_clean['Customer ID'].astype(int).astype(str)
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])


# 2. Total Price Calculation (THE FIX)
# We multiply Clean Quantity by Clean Price

In [28]:
df_clean['Total Price'] = df_clean['Quantity'] * df_clean['Price']

In [29]:
print(f"Data Cleaned. Rows: {df_clean.shape[0]}")
df_clean.head()

Data Cleaned. Rows: 407664


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total Price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


In [30]:
save_path = '../data/processed/online_retail_clean.csv'
df_clean.to_csv(save_path,index=False)
print(f'cleaned dataset ')

cleaned dataset 


In [31]:
refrence_date = df_clean['InvoiceDate'].max() + pd.DateOffset(days=1)

# Group by Customer ID


In [32]:
rfm = df_clean.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (refrence_date - x.max()).days, # Recency
    'Invoice': 'nunique', # Frequency
    'Total Price': 'sum'  # Monetary (Summing the calculated Total Price)
}).reset_index()

# Rename columns


In [33]:
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'Invoice': 'Frequency',
    'Total Price': 'Monetary'
}, inplace=True)

In [34]:
print(rfm.head())

  Customer ID  Recency  Frequency  Monetary
0       12346      165         11    372.86
1       12347        3          2   1323.32
2       12348       74          1    222.16
3       12349       43          3   2671.14
4       12351       11          1    300.93
